# Data Re-Uploading QCNN_TTN

This demo uses 4-qubit Quantum Convolutional Neural Network (QCNN) to see how pre-training the quantum embedding can be helpful for training a parameterized QML circuits for classfication tasks.

If you are interested in detailed analysis of pre-training on 8-qubit device, check out "/Result/earlystop 10 experiments/" folder.

If you are interested in the details about the QCNN used in this demo, check out https://arxiv.org/pdf/2108.00661.pdf.


In [1]:
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, '/home/tak/Github/QEmbedding/')
import torch
from torch import nn
import data
import embedding

2024-03-24 13:15:09.929850: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


IBMQ Fake provider

In [2]:
import pennylane as qml
from pennylane import numpy as np
from qiskit import IBMQ
from qiskit.providers.fake_provider import FakeJakarta, FakeMontreal
from qiskit_aer.noise import NoiseModel


noisy = FakeJakarta()
noise_model = NoiseModel.from_backend(noisy)
coupling_map = noisy.configuration().coupling_map
basis_gates = noise_model.basis_gates

dev_fake = qml.device(
    'qiskit.aer',
    wires=4,
    shots=1024,
    noise_model=noise_model,
    coupling_map=coupling_map,
    basis_gates=basis_gates
)

# Part1: Noisy Model1 & Model 2

Load the dataset for Model1 and Model2

In [3]:
dev = qml.device('default.qubit', wires=4)
feature_reduction = 'PCA4'
classes = [0,1]
X_train, X_test, Y_train, Y_test = data.data_load_and_process('mnist', feature_reduction=feature_reduction, classes=classes)

2024-03-24 13:15:16.950227: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


## Part1: Pre-training the embedding

In [4]:
steps = 1000
learning_rate = 0.01
batch_size = 25
ansatz = 'TTN'

def data_reuploading_ansatz(params): #8 params
    for i in range(4):
        qml.RY(params[i], wires=i)
    for i in range(3):
        qml.IsingZZ(params[i+4], wires=[i,i+1])
    qml.IsingZZ(params[7], wires=[3,0])

def data_reuploading_embedding(num_layers, params, x):
    for l in range(num_layers):
        data_reuploading_ansatz(params[8 * l : 8 * (l + 1)])
        embedding.Noisy_Four_QuantumEmbedding1(x)


@qml.qnode(dev_fake)
def data_reuploading_QCNN_classifier(num_layers, params, x):
    data_reuploading_embedding(num_layers, params, x)
    embedding.Noisy_QCNN_four(params[8 * num_layers: 8 * num_layers + 4], ansatz)
    return qml.expval(qml.PauliZ(2))


def Linear_Loss(labels, predictions):
    loss = 0
    for l,p in zip(labels, predictions):
        loss += 0.5 * (1 - l * p)
    return loss / len(labels)


def cost(weights, X_batch, Y_batch, num_layers):
    preds = [data_reuploading_QCNN_classifier(num_layers, weights, x) for x in X_batch]
    return Linear_Loss(Y_batch, preds)


def circuit_training(X_train, Y_train, num_layers):

    if ansatz == 'SU4':
        num_weights = (num_layers * 8) + 30
    elif ansatz == 'TTN':
        num_weights = (num_layers * 8) + 4

    weights = np.random.random(num_weights, requires_grad = True)
    opt = qml.NesterovMomentumOptimizer(stepsize=learning_rate)
    loss_history = []
    for it in range(steps):
        batch_index = np.random.randint(0, len(X_train), (batch_size,))
        X_batch = [X_train[i] for i in batch_index]
        Y_batch = [Y_train[i] for i in batch_index]
        weights, cost_new = opt.step_and_cost(lambda v: cost(v, X_batch, Y_batch, num_layers),
                                                     weights)
        loss_history.append(cost_new)
        if it % 200 == 0:
            print("iteration: ", it, " cost: ", cost_new)
    return loss_history, weights

In [6]:

Loss_histories_L1, weights_L1, Loss_histories_L2, weights_L2, Loss_histories_L3, weights_L3 = [], [], [], [], [], []
for i in range(5):
    loss_L1, weight_L1 = circuit_training(X_train, Y_train, 1)
    loss_L2, weight_L2 = circuit_training(X_train, Y_train, 2)
    loss_L3, weight_L3 = circuit_training(X_train, Y_train, 3)

    Loss_histories_L1.append(loss_L1)
    Loss_histories_L2.append(loss_L2)
    Loss_histories_L3.append(loss_L3)

    weights_L1.append(weight_L1)
    weights_L2.append(weight_L2)
    weights_L3.append(weight_L3)

f = open('/home/tak/Github/QEmbedding/Results/QCNN_demonstration/Noisy/Loss_histories_and_weights_data_reuploading.txt', 'a')
for i in range(5):
    f.write(f'Loss History L=1 {i + 1}:')
    f.write('\n')
    f.write(str(Loss_histories_L1[i]))
    f.write('\n')
for i in range(5):
    f.write(f'Weights L1 {i + 1}:')
    f.write('\n')
    f.write(str(weights_L1[i]))


for i in range(5):
    f.write(f'Loss History L=2 {i + 1}:')
    f.write('\n')
    f.write(str(Loss_histories_L2[i]))
    f.write('\n')
for i in range(5):
    f.write(f'Weights L=2 {i + 1}:')
    f.write('\n')
    f.write(str(weights_L2[i]))

for i in range(5):
    f.write(f'Loss History L=3 {i + 1}:')
    f.write('\n')
    f.write(str(Loss_histories_L3[i]))
    f.write('\n')
for i in range(5):
    f.write(f'Weights L=3 {i + 1}:')
    f.write('\n')
    f.write(str(weights_L3[i]))
f.close()



iteration:  0  cost:  0.4592578125
iteration:  200  cost:  0.410234375
iteration:  400  cost:  0.3971875
iteration:  600  cost:  0.3321875
iteration:  800  cost:  0.3296484375


# Part 4: Result 

Load the results

In [ ]:
PATH_LH0 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noisy/Loss_histories_not_trained.txt'
PATH_LH1 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noisy/Loss_histories_Model1.txt'
PATH_LH2 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noisy/Loss_histories_Model2.txt'
PATH_LH3 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noisy/Loss_histories_Model3.txt'
PATH_LH_amp0 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noisy/Loss_histories_Model_amp_not_trained.txt'
PATH_LH_amp1 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noisy/Loss_histories_Model_amp.txt'

PATH_w0 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noisy/weights_not_trained.txt'
PATH_w1 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noisy/weights_Model1.txt'
PATH_w2 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noisy/weights_Model2.txt'
PATH_w3 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noisy/weights_Model3.txt'
PATH_w_amp0 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noisy/weights_Model_amp_not_trained.txt'
PATH_w_amp1 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Noisy/weights_Model_amp.txt'

def data_read(PATH):
  textfile = open(PATH, 'r')
  data = np.array([])
  for line in textfile:
    a = np.array([float(v) for v in line.strip().split(" ")])
    data = np.append(data, a)
  return data

Loss_histories_not_trained = data_read(PATH_LH0).reshape(5,1000)
Loss_histories_Model1_Fidelity = data_read(PATH_LH1).reshape(5,1000)
Loss_histories_Model2_Fidelity = data_read(PATH_LH2).reshape(5,1000)
Loss_histories_Model3_Fidelity = data_read(PATH_LH3).reshape(5,1000)
Loss_histories_Model_amp_not_trained = data_read(PATH_LH_amp0).reshape(5,1000)
Loss_histories_Model_amp = data_read(PATH_LH_amp1).reshape(5,1000)

weights_not_trained = data_read(PATH_w0).reshape(5,4)
weights_Model1_Fidelity = data_read(PATH_w1).reshape(5,4)
weights_Model2_Fidelity = data_read(PATH_w2).reshape(5,4)
weights_Model3_Fidelity = data_read(PATH_w3).reshape(5,4)
weights_Model_amp_not_trained = data_read(PATH_w_amp0).reshape(5,4)
weights_Model_amp = data_read(PATH_w_amp1).reshape(5,4)

Not_trained_mean, Not_trained_std = Loss_histories_not_trained.mean(axis=0), Loss_histories_not_trained.std(axis=0)
Model1_Fidelity_mean, Model1_Fidelity_std = Loss_histories_Model1_Fidelity.mean(axis=0), Loss_histories_Model1_Fidelity.std(axis=0)
Model2_Fidelity_mean, Model2_Fidelity_std = Loss_histories_Model2_Fidelity.mean(axis=0), Loss_histories_Model2_Fidelity.std(axis=0)
Model3_Fidelity_mean, Model3_Fidelity_std = Loss_histories_Model3_Fidelity.mean(axis=0), Loss_histories_Model3_Fidelity.std(axis=0)
Model_amp_not_trained_mean, Model_amp_not_trained_std = Loss_histories_Model_amp_not_trained.mean(axis=0), Loss_histories_Model_amp_not_trained.std(axis=0)
Model_amp_mean, Model_amp_std = Loss_histories_Model_amp.mean(axis=0), Loss_histories_Model_amp.std(axis=0)

Plot the results for Model 1,2,3

In [ ]:
import seaborn as sns

plt.rcParams['figure.figsize'] = [10, 5]
fig, ax = plt.subplots()
clrs = sns.color_palette("husl", 4)
with sns.axes_style("darkgrid"):
    ax.plot(range(len(Not_trained_mean)), Not_trained_mean, label="No Pre-training", c=clrs[0])
    ax.fill_between(range(len(Not_trained_mean)), Not_trained_mean-Not_trained_std, Not_trained_mean+Not_trained_std, alpha=0.3,facecolor=clrs[0])

    ax.plot(range(len(Model1_Fidelity_mean)), Model1_Fidelity_mean, label="Noisy Model1 Pre-training", c=clrs[1])
    ax.fill_between(range(len(Model1_Fidelity_mean)), Model1_Fidelity_mean-Model1_Fidelity_std, Model1_Fidelity_mean+Model1_Fidelity_std, alpha=0.3,facecolor=clrs[1])
    
    ax.plot(range(len(Model2_Fidelity_mean)), Model2_Fidelity_mean, label="Noisy Model2 Pre-training", c=clrs[2])
    ax.fill_between(range(len(Model2_Fidelity_mean)), Model2_Fidelity_mean-Model2_Fidelity_std, Model2_Fidelity_mean+Model2_Fidelity_std, alpha=0.3,facecolor=clrs[2])

    ax.plot(range(len(Model3_Fidelity_mean)), Model3_Fidelity_mean, label="Noisy Model2 Pre-training", c=clrs[3])
    ax.fill_between(range(len(Model3_Fidelity_mean)), Model3_Fidelity_mean-Model3_Fidelity_std, Model3_Fidelity_mean+Model3_Fidelity_std, alpha=0.3,facecolor=clrs[3])


    ax.plot(range(1000), np.ones(1000) * LB_before_traindata, linestyle='dashed', linewidth=1.5, label="Lower Bound without Pre-training", c=clrs[0])
    ax.plot(range(1000), np.ones(1000) * LB_Model1_traindata, linestyle='dashed', linewidth=1.5, label="Lower Bound with Noisy Model1", c=clrs[1])
    ax.plot(range(1000), np.ones(1000) * LB_Model2_traindata, linestyle='dashed', linewidth=1.5, label="Lower Bound with Noisy Model2", c=clrs[2])
    ax.plot(range(1000), np.ones(1000) * LB_Model3_traindata, linestyle='dashed', linewidth=1.5, label="Lower Bound with Noisy Model3", c=clrs[3])


ax.set_xlabel("Iteration")
ax.set_ylabel("Loss")
ax.set_title("Noisy QCNN Loss History")
ax.legend()

Plot the results for Model Amplitude

In [ ]:
LB_amp_before_traindata = 0.2314
LB_amp_after_traindata = 0.0266
import seaborn as sns

plt.rcParams['figure.figsize'] = [10, 5]
fig, ax = plt.subplots()
clrs = sns.color_palette("husl", 2)
with sns.axes_style("darkgrid"):
    ax.plot(range(len(Model_amp_not_trained_mean)), Model_amp_not_trained_mean, label="No Pre-training", c=clrs[0])
    ax.fill_between(range(len(Model_amp_not_trained_mean)), Model_amp_not_trained_mean-Model_amp_not_trained_std, Model_amp_not_trained_mean+Model_amp_not_trained_std, alpha=0.3,facecolor=clrs[0])

    ax.plot(range(len(Model_amp_mean)), Model_amp_mean, label="With Pre-training", c=clrs[1])
    ax.fill_between(range(len(Model_amp_mean)), Model_amp_mean-Model_amp_std, Model_amp_mean+Model_amp_std, alpha=0.3,facecolor=clrs[1])


    ax.plot(range(1000), np.ones(1000) * LB_amp_before_traindata, linestyle='dashed', linewidth=1.5, label="Lower Bound without Pre-training", c=clrs[0])
    ax.plot(range(1000), np.ones(1000) * LB_amp_after_traindata, linestyle='dashed', linewidth=1.5, label="Lower Bound with Pre-training", c=clrs[1])


ax.set_xlabel("Iteration")
ax.set_ylabel("Loss")
ax.set_title("Noisy QCNN Loss History with Amplitude Embedding")
ax.legend()

Calculate the classification accuracies for Model1,2

In [ ]:
feature_reduction = 'PCA4'
X_train, X_test, Y_train, Y_test = data.data_load_and_process('mnist', feature_reduction=feature_reduction, classes=classes)

X_train, X_test = torch.tensor(X_train).to(torch.float32), torch.tensor(X_test).to(torch.float32)
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]

In [ ]:
def accuracy_test(predictions, labels):
    acc = 0
    for l, p in zip(labels, predictions):
        if np.abs(l - p) < 1:
            acc = acc + 1
    return acc / len(labels)


accuracies_not_trained, accuracies_Model1_Fidelity, accuracies_Model2_Fidelity = [], [], []

for i in range(5):
    prediction_not_trained = [Noisy_QCNN_classifier(weights_not_trained[i], x, Trained=False) for x in X_test]
    prediction_Model1_Fidelity = [Noisy_QCNN_classifier(weights_Model1_Fidelity[i], x, Trained='Model1_Fidelity') for x in X_test]
    prediction_Model2_Fidelity = [Noisy_QCNN_classifier(weights_Model2_Fidelity[i], x, Trained='Model2_Fidelity') for x in X_test]
    
    accuracy_not_trained = accuracy_test(prediction_not_trained, Y_test)
    accuracy_Model1_Fidelity = accuracy_test(prediction_Model1_Fidelity, Y_test)
    accuracy_Model2_Fidelity = accuracy_test(prediction_Model2_Fidelity, Y_test)

    accuracies_not_trained.append(accuracy_not_trained)
    accuracies_Model1_Fidelity.append(accuracy_Model1_Fidelity)
    accuracies_Model2_Fidelity.append(accuracy_Model2_Fidelity)

accuracies_not_trained = np.array(accuracies_not_trained)
accuracies_Model1_Fidelity, accuracies_Model2_Fidelity = np.array(accuracies_Model1_Fidelity), np.array(accuracies_Model2_Fidelity)

Calculate the accuracies for Model 3

In [ ]:
feature_reduction = False
X_train, X_test, Y_train, Y_test = data.data_load_and_process('mnist', feature_reduction=feature_reduction, classes=classes)

X_train, X_test = torch.tensor(X_train).to(torch.float32), torch.tensor(X_test).to(torch.float32)
X_train, X_test = X_train.permute(0, 3, 1, 2), X_test.permute(0, 3, 1, 2)
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]

In [ ]:
accuracies_Model3_Fidelity = []

for i in range(5):
    prediction_Model3_Fidelity = [Noisy_QCNN_classifier(weights_Model3_Fidelity[i], x, Trained='Model3_Fidelity') for x in X_test]
    accuracy_Model3_Fidelity = accuracy_test(prediction_Model3_Fidelity, Y_test)
    accuracies_Model3_Fidelity.append(accuracy_Model3_Fidelity)

accuracies_Model3_Fidelity =np.array(accuracies_Model3_Fidelity)

Calculate the accuracies for Model Amplitude

In [ ]:
feature_reduction = 'PCA16'
X_train, X_test, Y_train, Y_test = data.data_load_and_process('mnist', feature_reduction=feature_reduction, classes=classes)

X_train, X_test = torch.tensor(X_train).to(torch.float32), torch.tensor(X_test).to(torch.float32)
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]

In [ ]:
accuracies_amp_not_trained, accuracies_amp = [], []

for i in range(5):
    prediction_amp_not_trained = [Noisy_QCNN_classifier(weights_Model_amp_not_trained[i], x, Trained='Model_amp_not_trained') for x in X_test]
    prediction_amp = [Noisy_QCNN_classifier(weights_Model_amp[i], x, Trained='Model_amp') for x in X_test]
    

    accuracy_amp_not_trained = accuracy_test(prediction_amp_not_trained, Y_test)
    accuracy_amp = accuracy_test(prediction_amp, Y_test)

    accuracies_amp_not_trained.append(accuracy_amp_not_trained)
    accuracies_amp.append(accuracy_amp)


accuracies_amp_not_trained, accuracies_amp = np.array(accuracies_amp_not_trained), np.array(accuracies_amp)

In [ ]:

print("Noisy QCNN classification accuracies:\n")
print("-------------------------------------\n")
print(f" Accuracy without pre-training: {accuracies_not_trained.mean()} ± {accuracies_not_trained.std()}")
print(f" Accuracy after pre-training with Noisy_Model1_Fidelity: {accuracies_Model1_Fidelity.mean()} ± {accuracies_Model1_Fidelity.std()}")
print(f" Accuracy after pre-training with Noisy_Model2_Fidelity: {accuracies_Model2_Fidelity.mean()} ± {accuracies_Model2_Fidelity.std()}")
print(f" Accuracy after pre-training with Noisy_Model3_Fidelity: {accuracies_Model3_Fidelity.mean()} ± {accuracies_Model3_Fidelity.std()}")
print("\n")
print("\n")
print("Noisy QCNN classification accuracies (Amplitude Embedding):\n")
print("-------------------------------------\n")
print(f" Accuracy (Amplitude) without pre-training: {accuracies_amp_not_trained.mean()} ± {accuracies_amp_not_trained.std()}")
print(f" Accuracy (Amplitude) with pre-training: {accuracies_amp.mean()} ± {accuracies_amp.std()}")

